In [ ]:
import numpy as np # type: ignore
import pandas as pd # type: ignore

import pickle as pkl
import plotly.express as px # type: ignore
import plotly.graph_objects as go # type: ignore

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from StreamlitApp.functions.carga_dataframes import *
from ML.escalado_datos import *
from ML.gru_rnn import *
from StreamlitApp.passwords import pw

from sklearn.model_selection import train_test_split # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import GRU, Dense, Input, Dropout # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore

### RNN GRU

Para este modelo importamos el scaler guardado en escalado_datos.py.

Luego definimos varias funciones para que:

- se creen secuencias en los datos en base a la ventana temporal deseada
- se redimensionen esas secuencias adecuadamente
- se cree la RNN GRU adaptada a las secuencias. Hemos añadido algunas capas Dropout para evitar el over-fitting.
- se predigan los valores de demanda con 1-step y multi-step
- se muestren métricas y gráficos relativos al desempeño del modelo en cuestión (luego se implementarán en Streamlit)

Entrenamos un modelo con lookback de 30 días y 1000 epochs y lo guardamos en .pkl para usar desde Streamlit. También guardamos un dataframe con métricas y los gráficos de loss-mae para mostrar posteriormente.

---

In [ ]:
# Cargar el scaler
with open("../data/data_scaled/scalers/scaler_consumo_anio_DF_DEMANDA.pkl", "br") as file:
    scaler = pkl.load(file)

In [ ]:
# Función para crear secuencias
def crea_secuencias(dataframe, target_col, len_secuencia) -> tuple:
    X, y = [], []
    for i in range(len(dataframe) - len_secuencia):
        X.append(dataframe.iloc[i:i+len_secuencia].drop(columns=[target_col]).values) 
        y.append(dataframe.iloc[i+len_secuencia][target_col]) 
    return np.array(X), np.array(y)

In [ ]:
# Función para redimensionar las secuencias
def redimensiona(xtrain, xtest, ventana, n_features) -> str:
    xtrain = xtrain.reshape((xtrain.shape[0], ventana, n_features))  
    xtest = xtest.reshape((xtest.shape[0], ventana, n_features)) 
    return f"X_train shape: {xtrain.shape}, X_test shape: {xtest.shape}"

In [ ]:
# Construcción del modelo GRU con más capas. Este modelo me parece más realista
def crea_gru(input_shape, len_secuencia, xtrain, xtest, ytrain, ytest) -> tuple: #--  agregar metricas en un df
    model = Sequential([
        Input(shape=(len_secuencia, input_shape)),
        GRU(64, activation='tanh'),  
        Dropout(0.3), 
        Dense(1, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='linear') 
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    #early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    #history = model.fit(x=xtrain, y=ytrain, validation_data=(xtest, ytest), epochs=100, verbose=0, callbacks=[early_stopping])  2500 epochs sin early probar
    history = model.fit(x=xtrain, y=ytrain, validation_data=(xtest, ytest), epochs=500, verbose=0)
    
    with open(f"MODELS/GRU/gru_model{len_secuencia}.pkl", "bw") as file:
        pickle.dump(model,file)
        return model, history

In [ ]:
# Función para graficar loss y mae
def grafica_loss_mae(historial) -> None:
    fig = px.line(data_frame=historial.history, 
                  y=['loss', 'val_loss'], 
                  title='Función de pérdida (loss) basada en el MSE',
                  labels={'index': 'Época', 'value': 'Pérdida'})
    
    fig.update_layout(title_x=0.5, 
                      template="plotly_white",
                      legend_title_text="Variables")
    
    fig.for_each_trace(lambda x: x.update(name="Pérdida Entrenamiento" if x.name == "loss" else "Pérdida Validación"))
    return None

In [ ]:
# Función para predecir 1-step
def predice_1step(model, data, scaler, len_secuencia, num_dias) -> tuple:
    validation_predictions = []
    
    i = -num_dias
    while len(validation_predictions) < num_dias:
        p = model.predict(data[i].reshape(1, len_secuencia, data.shape[2]))[0, 0]
        validation_predictions.append(p)
        i += 1

    # Desescalado
    dummy_features = np.zeros((len(validation_predictions), 1))
    predictions_with_dummy = np.hstack([np.array(validation_predictions).reshape(-1, 1), dummy_features])
    predictions_desescalado = scaler.inverse_transform(predictions_with_dummy)[:, 0]
    
    return predictions_desescalado

In [ ]:
# Función para predecir multi-step
def predice_multistep(model, data, scaler, len_secuencia, num_dias) -> np.array:
    predictions = []
    input_seq = data[-1].reshape(1, len_secuencia, data.shape[2])
    
    for _ in range(num_dias):
        pred = model.predict(input_seq)[0, 0]
        predictions.append(pred)
        input_seq = np.roll(input_seq, shift=-1, axis=1)  
        input_seq[0, -1, -1] = pred 
    
    # Desescalado
    dummy_features = np.zeros((len(predictions), 1))
    predictions_with_dummy = np.hstack([np.array(predictions).reshape(-1, 1), dummy_features])
    predictions_desescalado = scaler.inverse_transform(predictions_with_dummy)[:, 0]

    return predictions_desescalado

In [ ]:
# Graficar las predicciones 1-step y multi-step
def grafica_predicciones(real, pred_1step, pred_multistep) -> None:
    fig = go.Figure()

    # Valores reales
    fig.add_trace(go.Scatter(
        y=real,
        mode='lines+markers',
        name='Valores Reales',
        line=dict(color='blue', width=2),
        marker=dict(size=6)
    ))

    # Predicciones 1-step
    fig.add_trace(go.Scatter(
        y=pred_1step,
        mode='lines+markers',
        name=f'Predicciones 1-step',
        line=dict(color='red', width=2, dash='dot'),
        marker=dict(size=6)
    ))

    # Predicciones multi-step
    fig.add_trace(go.Scatter(
        y=pred_multistep,
        mode='lines+markers',
        name=f'Predicciones Multi-step',
        line=dict(color='green', width=2, dash='dot'),
        marker=dict(size=6)
    ))

    # Configuración de la gráfica
    fig.update_layout(
        title="Predicciones vs Valores Reales",
        title_x=0.5,
        xaxis_title="Días",
        yaxis_title="Demanda (GWh)",
        template="plotly_white",
        hovermode="x",
        xaxis=dict(
            tickvals=list(range(len(pred_1step) + 1)),  
            ticktext=[str(i) for i in range(1, len(pred_1step) + 1)]  
        ))

    fig.show()
    return None

In [ ]:
# Graficar las predicciones futuras
def grafica_predicciones_futuras(pred_futuro) -> None:
    fig = go.Figure()

    # Predicciones futuras
    fig.add_trace(go.Scatter(
        y=pred_futuro,
        mode='lines+markers',
        name=f'Predicciones Futuras',
        line=dict(color='orange', width=2, dash='dot'),
        marker=dict(size=6)
    ))

    # Configuración de la gráfica
    fig.update_layout(
        title="Predicciones para Días Futuros",
        title_x=0.5,
        xaxis_title="Días Futuros",
        yaxis_title="Demanda (GWh)",
        template="plotly_white",
        hovermode="x",
        xaxis=dict(
            tickvals=list(range(len(pred_futuro))),
            ticktext=[str(i + 1) for i in range(len(pred_futuro))]
        ))

    fig.show()
    return None

#### Código usado para entrenar los modelos en local y guardalos para su posterior uso en Streamlit con archivo gru_rnn.py

---

In [ ]:
df_demanda = carga_dataframes(pw["host"], pw["user"], pw["password"], pw["database"])[1]

# df_filtrado simula la función vis_demanda que se usa en main.py, que filtra los datos y luego se usan para las predicciones
df_filtrado = df_demanda.copy()
df_filtrado['valor_(GWh)'] = df_filtrado['valor_(MWh)'].apply(lambda x : x * 0.001)
df_filtrado = df_filtrado[df_filtrado["titulo"] == "Demanda"]

df = procesar_datos(df_demanda)
df = df.drop(columns="fecha")
TARGET = df["valor_(GWh)"]

# Se usaron ventanas de 7, 15 y 30 días
ventana = 7  

n_features = len([col for col in df.columns if col != TARGET.name])
X, y = crea_secuencias(df, TARGET.name, ventana)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)  # No barajar los datos
redimensiona(X_train, X_test, ventana, n_features)

model, history = crea_gru(X.shape[2], ventana, X_train, X_test, y_train, y_test)
grafica_loss_mae(history)

pred_1step, real_1step = predice_1step(model, X_test, y_test, scaler, ventana, num_dias=ventana)
pred_multistep = predice_multistep(model, X_test, scaler, ventana, num_dias_multi=ventana)

metricas_1step = muestra_metricas(df_filtrado, ventana, pred_1step)
metricas_multistep = muestra_metricas(df_filtrado, ventana, pred_multistep)

grafico_mae = grafica_loss_mae(history)
grafico_mae.write_image(f"MODELS/GRU/Grafico LOSS_MAE_{ventana}.png")
grafica_predicciones(real_1step, pred_1step, pred_multistep)

metricas = {
"modelo" : f"GRUmodel{ventana}",
"prediccion": ["1-step", "multi-step"],
"r2": [metricas_1step["r2"], metricas_multistep["r2"]],
"mae_GWh": [metricas_1step["mae_GWh"], metricas_multistep["mae_GWh"]],
"rmse_GWh": [metricas_1step["rmse_GWh"], metricas_multistep["rmse_GWh"]]
}

df_metricas = pd.read_csv("MODELS/GRU/MetricasGRU.csv")
df_metricas_ventana = pd.DataFrame(metricas)
df_metricas = pd.concat([df_metricas, df_metricas_ventana])

df_metricas.to_csv("MODELS/GRU/MetricasGRU.csv", index=False)